In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
using LinearAlgebra, Plots
import FiniteDiff
import ForwardDiff as FD
using MeshCat
using Test
using Plots

include(joinpath(@__DIR__, "utils", "fmincon.jl"))

  Activating project at `c:\Users\aphia\Desktop\school\ocrl\QuadraCat`


fmincon (generic function with 1 method)

In [2]:
include(joinpath(@__DIR__, "utils", "quadruped_new.jl"))
model = UnitreeA1()
@show num_positions(model.mech)
@show num_velocities(model.mech)
@show joints(model.mech)
@show bodies(model.mech)

num_positions(model.mech) = 19
num_velocities(model.mech) = 18
joints(model.mech) = Joint{Float64, JT} where JT<:JointType{Float64}[Joint "base_to_world": Quaternion floating joint, Joint "floating_base": Fixed joint, Joint "imu_joint": Fixed joint, Joint "FR_hip_joint": Revolute joint with axis [1.0, 0.0, 0.0], Joint "FL_hip_joint": Revolute joint with axis [1.0, 0.0, 0.0], Joint "RR_hip_joint": Revolute joint with axis [1.0, 0.0, 0.0], Joint "RL_hip_joint": Revolute joint with axis [1.0, 0.0, 0.0], Joint "FR_thigh_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "FL_thigh_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "RR_thigh_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "RL_thigh_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "FR_calf_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "FL_calf_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "RR_calf_joint": Revolute joint with axis [0.0, 1.0, 0.0], Joint "RL_calf_joint": Revolute joi

20-element Vector{RigidBody{Float64}}:
 RigidBody: "world"
 RigidBody: "base"
 RigidBody: "trunk"
 RigidBody: "imu_link"
 RigidBody: "FR_hip"
 RigidBody: "FL_hip"
 RigidBody: "RR_hip"
 RigidBody: "RL_hip"
 RigidBody: "FR_thigh"
 RigidBody: "FL_thigh"
 RigidBody: "RR_thigh"
 RigidBody: "RL_thigh"
 RigidBody: "FR_calf"
 RigidBody: "FL_calf"
 RigidBody: "RR_calf"
 RigidBody: "RL_calf"
 RigidBody: "FR_foot"
 RigidBody: "FL_foot"
 RigidBody: "RR_foot"
 RigidBody: "RL_foot"

In [3]:
xic = initial_state(model)
q = xic[1:num_positions(model.mech)]

@show get_trunk_position(model, q)
@show get_foot_position(model, q, "RR")
@show get_foot_position(model, q, "FL")
@show get_trunk_velocity(model, xic)

get_trunk_position(model, q) = [0.0, 0.0, 0.2571150438746157]
get_foot_position(model, q, "RR") = [-0.183, -0.13205, -2.7755575615628914e-17]
get_foot_position(model, q, "FL") = [0.183, 0.13205, -2.7755575615628914e-17]
get_trunk_velocity(model, xic) = [0.0, 0.0, 0.0]


3-element SVector{3, Float64} with indices SOneTo(3):
 0.0
 0.0
 0.0

In [4]:
# visualize initial state
mvis = initialize_visualizer(model)
xic = initial_state(model)
set_configuration!(mvis, xic[1:num_positions(model.mech)])
render(mvis)

┌ Info: Listening on: 127.0.0.1:8700, thread id: 1
└ @ HTTP.Servers C:\Users\aphia\.julia\packages\HTTP\MIUdD\src\Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat C:\Users\aphia\.julia\packages\MeshCat\9QrxD\src\visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("block" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x62, 0x6c, 0x6f, 0x63, 0x6b], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x62, 0x6c, 0x6f, 0x63, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x77, 0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("trunk" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x62, 0x61, 0x73, 0x65, 0x2f, 0x74, 0x72, 0x75, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_floating_base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x61, 0x74, 0x69, 0x6e, 0x67, 0x5f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_hip" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x6e, 0x6b, 0x2f, 0x52, 0x4c, 0x5f, 0x68, 0x69, 0x70], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_thigh" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x2f, 0x52, 0x4c, 0x5f, 0x74, 0x68, 0x69, 0x67, 0x68], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_thigh_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x69, 0x67, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_17" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_calf" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x68, 0x2f, 0x52, 0x4c, 0x5f, 0x63, 0x61, 0x6c, 0x66], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_foot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x66, 0x2f, 0x52, 0x4c, 0x5f, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_foot_fixed" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x66, 0x69, 0x78, 0x65, 0x64], Dict{String, Vector{UInt8}}(), not

In [5]:
## set up discrete time dynamics

# our continuous time dynamics function is xdot = dynamics(model, x, u)

function hermite_simpson_ground(params::NamedTuple, x1::Vector, x2::Vector, u_hat::Vector, dt)::Vector
    model = params.model
    u = u_hat[1:end-4]
    λ = u_hat[end-3:end]
    
    x12 = 0.5*(x1+x2) + dt/8*(dynamics(model, x1, u, λ) - dynamics(model, x2, u, λ))
    return x1 + dt/6 * (dynamics(model, x1, u, λ) + 4 * dynamics(model, x12, u, λ) + dynamics(model, x2, u, λ)) - x2
end

hermite_simpson_ground (generic function with 1 method)

In [6]:
## set up cost

function quadracat_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref,Uref = params.Xref, params.Uref

    J = 0
    for i=1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        J += 0.5*(xi - Xref[i])'*Q*(xi - Xref[i]) + 0.5*(ui - Uref[i])'*R*(ui - Uref[i])
    end
    xN = Z[idx.x[N]]
    J += 0.5*(xN - Xref[N])'*Qf*(xN - Xref[N])
        
    return J

end

quadracat_cost (generic function with 1 method)

In [7]:
## set up equality constraints 

function quadracat_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt

    c = zeros(eltype(Z), idx.nc)

    for i=1:(N-1)
        xk = Z[idx.x[i]]
        uk = Z[idx.u[i]]
        xk1 = Z[idx.x[i+1]]

        c[idx.c[i]] = hermite_simpson_ground(params, xk, xk1, uk, dt)
    end
    return c 
end

function quadracat_stance_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, model = params.idx, params.N, params.model
    nq = params.nq
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 0)
    
    # each foot is pinned to the ground
    for i=1:(N-1)
        xk = Z[idx.x[i]]
        v = xk[nq+1:end]
        # c = vcat(c, [
        #                     get_foot_position(model, q, "RR")[3]; 
        #                     get_foot_position(model, q, "RL")[3];
        #                     get_foot_position(model, q, "FL")[3];
        #                     get_foot_position(model, q, "FR")[3]
        #                     ])
        c = vcat(c, [
            (jac_foot(model, q, "RR")*v)[3];
            (jac_foot(model, q, "RL")*v)[3];
            (jac_foot(model, q, "FL")*v)[3];
            (jac_foot(model, q, "FR")*v)[3]
        ])
    end

    return c

end

function quadracat_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic = params.N, params.idx, params.xic
    xg = params.xg
    
    # TODO: stack up all of our equality constraints 
    
    # should be length 2*nx + (N-1)*nx + N 
    # inital condition constraint (nx)       (constraint 1)
    # terminal constraint         (nx)       (constraint 2)
    # dynamics constraints        (N-1)*nx   (constraint 3-6)
    # stance constraint           N          (constraint 7-8)
    ic_con = Z[idx.x[1]] - xic
    xg_con = Z[idx.x[N]] - xg
    return [ic_con; 
    xg_con;
    quadracat_dynamics_constraints(params, Z); 
    quadracat_stance_constraint(params, Z)
    ]
end

quadracat_equality_constraint (generic function with 1 method)

In [ ]:
## set up inequality constraint 

function quadracat_inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, model = params.idx, params.N, params.model
    xg = params.xg
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 0)
    
    # for i=1:(N-1)
    #     xk = Z[idx.x[i]]
    #     v = xk[nq+1:end]
    #     c = vcat(c, [
    #         (jac_foot(model, q, "RR")*v)[3];
    #         (jac_foot(model, q, "RL")*v)[3];
    #         (jac_foot(model, q, "FL")*v)[3];
    #         (jac_foot(model, q, "FR")*v)[3]
    #     ])
    # end

    c = vcat(c, norm(Z[idx.x[N]] - xg, Inf))

    return c
end

In [8]:
function solve_quadracat_crouch(;verbose=true)
    
    # problem size 
    nq = num_positions(model.mech)
    nv = num_velocities(model.mech)
    nx = nq+nv
    nu = control_dim(model)+4 # stack normal forces at feet at the end of the control vector
    
    #Xref, Uref, t_vec = reference_trajectory(model)
    #N = length(t_vec)
    tf = 0.5
    dt = 0.25
    t_vec = 0:dt:tf 
    N = length(t_vec)
    Xref = [initial_state(model) for i=1:N]
    Uref = [0.001*randn(nu) for i=1:(N-1)]
    
    # LQR cost 
    Q = diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*deepcopy(Q)
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial and goal states 
    xic = Xref[1]
    xg = Xref[end]

    # initial guess 
    z0 = zeros(idx.nz)
    for i=1:(N-1)
        z0[idx.x[i]] = Xref[i]
        z0[idx.u[i]] = Uref[i]
    end
    z0[idx.x[N]] = Xref[N]

    # load all useful things into params 
    params = (
        model=model,
        nx=nx, 
        nu=nu, 
        nq=nq,
        nv=nv,
        dt=dt,
        Q = Q, 
        R = R, 
        Qf = Qf, 
        xic = xic, 
        xg=xg,
        N = N, 
        idx = idx,
        g = 9.81,
        Xref=Xref,
        Uref=Uref
        )
    
    # TODO: primal bounds 
    x_l = -Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:(N-1)
        x_l[idx.u[i][end-3:end]] = [0; 0; 0; 0]
    end
    
    # inequality constraint bounds
    c_l = zeros(0)
    c_u = zeros(0)
    function quadracat_inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end
    # c_l = zeros(4*(N-1))
    # c_u = Inf*ones(4*(N-1))
    # c_l = -Inf*ones(1)
    # c_u = 1e-2*ones(1)

    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    
    Z = fmincon(quadracat_cost,quadracat_equality_constraint,quadracat_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 1_00, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i][1:end-4]] for i = 1:(N-1)]
    λ = [Z[idx.u[i][end-3:end]] for i = 1:(N-1)]
    
    return X, U, λ, t_vec, params 
end

solve_quadracat_crouch (generic function with 1 method)

In [9]:
X, U, λ, t_vec, params_dircol = solve_quadracat_crouch(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    22308
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian..

([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2571150438746157, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2571150438746157, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2571150438746157, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0010716451739527923, 0.00022178932990277582, -0.0007069601122874432, -0.0010414112651248159, -0.00038547592294858917, 0.0015599135383077515, 0.0002549837655025387, -0.0017942302592989477, 0.0009273378452333733, 0.00232222286935268, 0.00029362123115416524, 8.58211547061595e-5], [-0.0009338919954614165, -0.0004335443858093089, -2.2663382287361536e-5, 0.0009995669988619492, 0.00047142606202405695, 0.0017197699628307616, 0.00023769539892420718, -0.0011830595012439174, 0.002396729903442467, 0.0003949566469809054, 0.0010334929615269538, 0.0007131802280928242]], [[-0.0004505908881940552, 0.0002638274931451017, 0.001216901740616164, 

In [ ]:
qs = [xk[1:num_positions(model.mech)] for xk in X] # extract positions from state vector

mvis = initialize_visualizer(model)
xic = initial_state(model)
set_configuration!(mvis, xic[1:num_positions(model.mech)])
anim = MeshCat.Animation(mvis, t_vec, qs)
setanimation!(mvis, anim)
render(mvis)

In [ ]:
nq = params_dircol.nq
# ------------plotting--------------
Xm = hcat(X...)
Um = hcat(U...)
λm = hcat(λ...)

display(plot(t_vec, Xm[1:7,:]', xlabel="Time Step", ylabel="q", title="Floating Body Position"))
display(plot(t_vec, Xm[8:nq,:]', xlabel="Time Step", ylabel="q", title="Joint Angles"))
display(plot(t_vec, Xm[nq+1:nq+6,:]', xlabel="Time Step", ylabel="v", title="Floating Body Velocity"))
display(plot(t_vec, Xm[nq+7:end,:]', xlabel="Time Step", ylabel="v", title="Joint Velocities"))
display(plot(t_vec[1:end-1], Um', xlabel="Time Step", ylabel="u", title="Controls"))
display(plot(t_vec[1:end-1], λm', xlabel="Time Step", ylabel="u", title="Contact Forces"))